In [1]:
import time
início=time.time()

In [2]:
import numpy as np
import pandas as pd
from pandas.tseries.offsets import DateOffset
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
comex_df_b=pd.read_csv('/home/andre301267/git/BD/temp/imp_fert_ncm_quanti.csv').sort_values(by=['CO_ANO','CO_MES'])

In [3]:
comex_df_b.insert(0,'date',pd.to_datetime(comex_df_b.CO_ANO.astype(str)+'/'+comex_df_b.CO_MES.astype(str)+'/1'))

In [4]:
qq=pd.DataFrame()

In [5]:
for xx in range(0,5*12):
    dm=comex_df_b.date.max() - DateOffset(months=xx)
    comex_df=comex_df_b[comex_df_b.date<=dm]

    data_ini=comex_df.date.min()
    data_fim=comex_df.date.max()
    
    # 1a Fase: Seleciona a planilha de interesse
    
    comex_df.nick.unique()
    
    nick=['MOP']
    UF=['BA']
    CO_PAIS=[]
    CO_URF=[]
    
    l=['nick','UF','CO_PAIS','CO_URF']
    d={'nick':nick,'UF':UF,'CO_PAIS':CO_PAIS,'CO_URF':CO_URF}
    for x in l:
        if len(d[x])>0:
            comex_df=comex_df[comex_df[x].isin(d[x])]
        else:
            comex_df.drop(columns=x,inplace=True)
    
    '''sns.displot(data=comex_df,x='KG_LIQUIDO',hue='km', log_scale=True,height=2,aspect=4)\
    .fig.suptitle(f"Importações de {nick[0]} em {UF[0]}",y=1.1)
    plt.show()'''
    
    km=2
    
    comex_df=comex_df[comex_df.km==km]
    comex_df.drop(columns='km',inplace=True)
    
    # 2a Fase: Sparse Matrix - train
    
    col_ord=['CO_ANO','CO_MES']
    col_cat=[x for x in l if x in comex_df.columns.tolist()]
    col_quanti=['KG_LIQUIDO','VL_FOB','VL_FRETE']
    comex_df=comex_df[col_ord+col_cat+col_quanti].sort_values(by=col_ord)
    comex_df[col_ord+col_cat]=comex_df[col_ord+col_cat].astype('category')
    comex_df_g=comex_df.groupby(by=col_ord+col_cat,observed=False)[col_quanti].sum().reset_index()
    comex_df_g.insert(0,'date',pd.to_datetime(comex_df_g.CO_ANO.astype(str)+'/'+comex_df_g.CO_MES.astype(str)+'/1'))
    comex_df_g=comex_df_g[(comex_df_g.date>=data_ini)&(comex_df_g.date<=data_fim)].sort_values('date')
    train=comex_df_g
    
    # 3a Fase: train & test
    
    train_end=train.date.max()-DateOffset(years=1)
    test=train[train.date>train_end]
    train=train[train.date<=train_end]
    
    # 4a Fase: Dummies
    
    comex_df_o=pd.concat([train,test])
    comex_df_o.CO_MES=comex_df_o.CO_MES.astype('category')
    comex_df_o.drop(columns='CO_ANO',inplace=True)
    comex_df_o.date=comex_df_o.date.astype(int)
    comex_df_dumm=pd.get_dummies(comex_df_o, prefix_sep='~')
    
    # 4a fase: Classificação
    
    comex_df_dumm=comex_df_dumm.assign(oc=1)
    comex_df_dumm.loc[comex_df_dumm.KG_LIQUIDO==0,'oc']=0
    comex_df_dumm.select_dtypes(exclude='bool')
    train=comex_df_dumm[comex_df_dumm.date.astype('datetime64[ns]')<=train_end]
    test=comex_df_dumm[comex_df_dumm.date.astype('datetime64[ns]')>train_end]
    X_train=train[['date']+train.select_dtypes('bool').columns.tolist()]
    y_train=train['oc']
    X_test=test[['date']+test.select_dtypes('bool').columns.tolist()]
    
    from sklearn.ensemble import RandomForestClassifier as RFC
    RFC=RFC()
    RFC.fit(X_train,y_train)
    test_pred=pd.concat([X_test.reset_index(drop=True),pd.DataFrame({'oc_pred':RFC.predict(X_test)})],axis=1)
    
    test_pred=test_pred.merge(test.drop(columns=col_quanti))
    
    # 5a fase: Regression
    
    train=train[train.oc==1]
    
    test_pred=test.merge(test_pred)
    
    test_pred_r=test_pred[(test_pred.oc==1)&(test_pred.oc_pred==1)]
    
    X_train=pd.concat([train[['date']],train.select_dtypes('bool')],axis=1)
    
    y_train=train[col_quanti]
    
    X_test=pd.concat([test_pred_r[['date']],test_pred_r.select_dtypes('bool')],axis=1)
    
    from sklearn.ensemble import RandomForestRegressor as REG
    REG=REG(max_features=1, max_depth=3)
    
    from sklearn.linear_model import LinearRegression as LM
    LM=LM()
    from sklearn.preprocessing import PolynomialFeatures
    from sklearn.pipeline import make_pipeline
    polynomial_features = PolynomialFeatures(degree=1)
    REG = make_pipeline(polynomial_features, LM)
    
    REG.fit(X_train,y_train)
    X_test=pd.concat([X_test,pd.DataFrame(REG.predict(X_test),columns=col_quanti)],axis=1)
    
    for w in col_quanti:
        X_test=X_test.rename(columns={w:w+'_pred'})
    
    test_pred=test_pred.merge(X_test)
    
    # 6a fase: Score
    
    b=test_pred.select_dtypes('bool')
    o=test_pred.select_dtypes(exclude='bool')
    
    m=pd.from_dummies(b.loc[:,b.columns.str.contains('CO_MES')],sep='~')
    n=pd.from_dummies(b.loc[:,b.columns.str.contains('nick')],sep='~')
    u=pd.from_dummies(b.loc[:,b.columns.str.contains('UF')],sep='~')
    results=pd.concat([o,m,n,u],axis=1)
    results.date=results.date.astype('datetime64[ns]')
    
    date_results=results[['date']].drop_duplicates().assign(dist_prev=np.arange(1,len(results.date.unique())+1))
    
    results=date_results.merge(results)
    
    qq=pd.concat([qq,results])

In [8]:
qq.info()

<class 'pandas.core.frame.DataFrame'>
Index: 650 entries, 0 to 8
Data columns (total 27 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   date             650 non-null    datetime64[ns]
 1   dist_prev        650 non-null    int64         
 2   KG_LIQUIDO       650 non-null    float64       
 3   VL_FOB           650 non-null    float64       
 4   VL_FRETE         650 non-null    int64         
 5   oc               650 non-null    int64         
 6   oc_pred          650 non-null    int64         
 7   KG_LIQUIDO_pred  589 non-null    float64       
 8   VL_FOB_pred      589 non-null    float64       
 9   VL_FRETE_pred    589 non-null    float64       
 10  CO_MES           358 non-null    object        
 11  nick             358 non-null    object        
 12  UF               358 non-null    object        
 13  CO_MES~1         292 non-null    object        
 14  CO_MES~2         292 non-null    object        
 

In [6]:
from sklearn import metrics
for w in col_quanti:
    print('mean_absolute_percentage_error - ',w,':',round(metrics.mean_absolute_percentage_error(qq[w],qq[w+'_pred']),1))

ValueError: Input contains NaN.

In [ ]:
fim=time.time()
print(f"Duração: {round((fim-início)/60,2)} minutos.")